<a href="https://colab.research.google.com/github/Geuens/curso_ai/blob/main/Copy_of_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dotenv import load_dotenv
import os

# Cargar variables de entorno
load_dotenv()

from openai import OpenAI


# Configurar el motor de OpenAI
engine = "gpt-3.5-turbo"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_completion(prompt, temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=engine,
        messages=messages,
        temperature=temperature, # grado de creatividad del modelo
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # grado de creatividad del modelo
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message.content

ModuleNotFoundError: No module named 'dotenv'

## Hilo de la conversación
ChatGPT mantiene un registro de cada hilo de la conversación, de tal manera que podemos referirnos a datos que le hemos proporcionado anteriormente. Si cambiamos el hilo, necesitaríamos pasarle de nuevo todos los datos. Así es como funciona:

In [ ]:
messages =  [
{'role':'system', 'content':'Eres un famoso cómico.'},
{'role':'user', 'content':'Cuéntame un chiste'},
{'role':'assistant', 'content':'¿Cómo se mide la masa de seguidores de un influencer?'},
{'role':'user', 'content':'Ni idea'}  ]

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [
{'role':'system', 'content':'Eres un bot amigable.'},
{'role':'user', 'content':'Hola, me llamo Carlos'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

### este hilo es distinto del anterior

In [ ]:
messages =  [
{'role':'system', 'content':'Eres un bot simpático.'},
{'role':'user', 'content':'¿Cómo me llamo?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

### Tratándose del mismo hilo funcionará, ya que le estamos enviando todas la inforamción necesaria

In [ ]:
messages =  [
{'role':'system', 'content':'Eres un bot amigable.'},
{'role':'user', 'content':'Hola, me llamo Carlos'},
{'role':'assistant', 'content': "¡Hola Carlos! Mucho gusto en conocerte. ¿En qué puedo ayudarte hoy?"},
{'role':'user', 'content':'Si, ¿Cómo me llamo?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

## Podemos establecer y mantener un contexto

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)

### Damos instrucciones de contexto detalladas para que se comporte como queremos.
Es importante ser especifico/a. Por ejemplo prueba a sustituir la frase:
"en función de sus conocimientos le planteas uno o varios cursos del catálogo"
por:
"función de sus conocimientos le planteas unitinerario formativo"

Al no ser especificos ciñéndonos al catálogo, verás que se "inventa" (hallucination) un itinerario com sugerencia.

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
Eres un servicio que ofrece sugerencias de cursos de formación técnicos. \
Primero saludas al cliente, recolectas sus intereses, y en
función de sus conocimientos le planteas uno o varios cursos del catálogo \
basado en los cursos disponibles en el catálogo.
Preguntas al cliente cuándo desearía recibir el curso. \
Finalmente le informas de todos los detalles del curso.\
Asegúrate de clarificar la opciones disponibles y contenidos \
Identifica el curso del catálogo, área de interés e itinerario del catálogo.\
Respondes de maneta concisa, educada y amigable. \
Si el curso no aparece en el catálogo discúlpate y ofrece una alternativa \
El catálogo incluye \
Áreas: Python, AI, Seguridad y Java
Título: Fundamentos de Python, Requisitos: Nociones de programación, Duración: 30 horas, Precio: 700 EUR, Áreas: Python \
Título: Python Avanzado, Requisitos: Fundamentos de Python, Duración: 40 horas, Precio: 1500 EUR, Áreas: Python \
Título: Machine Learning con Python, Requisitos: Python Avanzado, Duración: 40 horas, Precio: 2500 EUR, Áreas: Python, AI \
Título: AI con Python, Requisitos: Machine Learning con Python, Duración: 60 horas, Precio: 3500 EUR, Áreas: Python, AI \
Título: Seguridad con Python, Requisitos; Python AVanzado, Duración: 30 horas, Precio: 1500 EUR, Áreas: Seguridad \
Título: Fundamentos de Java, Requisitos: Nociones de programación, Duración: 30 horas, Precio: 700 EUR, Áreas: Java \
Título: Java Avanzado, Requisitos: Fundamentos de Python, Duración: 40 horas, Precio: 1500 EUR  Áreas: Java  \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hola", placeholder='Introduce el texto…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard